In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from random import shuffle


In [2]:
train_dir = 'train_data'
df = pd.read_csv('train_labels.csv')
IMG_SIZE = 50
LR = 1e-3

In [3]:
def label_image(img):
    result = df.loc[df['id']==int(img)]['label']
    if result.tolist()[0] == 'cat':
        return 0
    else:
        return 1

In [4]:
def preprocess_training_data():
    training_data = []
    for img_path in os.listdir(train_dir):
        label = label_image(img_path.split('.')[0])
        image = cv2.resize(cv2.imread(os.path.join(train_dir,img_path),cv2.IMREAD_UNCHANGED),(IMG_SIZE,IMG_SIZE))
        training_data.append([image,label])
    shuffle(training_data)
    return training_data

In [5]:
training_data = preprocess_training_data()

In [6]:
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X)/255
y = np.array(y)


In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [8]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128,input_shape=X.shape[1:],activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(2, activation='softmax'))

In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(X,y,epochs=10,validation_split=0.1,batch_size=32)

Epoch 1/10
225/225 [==============================] - 21s 92ms/step - loss: 0.6845 - accuracy: 0.5517 - val_loss: 0.6637 - val_accuracy: 0.6025
Epoch 2/10
225/225 [==============================] - 19s 85ms/step - loss: 0.6426 - accuracy: 0.6343 - val_loss: 0.6118 - val_accuracy: 0.6562
Epoch 3/10
225/225 [==============================] - 21s 94ms/step - loss: 0.5934 - accuracy: 0.6786 - val_loss: 0.5785 - val_accuracy: 0.6963
Epoch 4/10
225/225 [==============================] - 18s 80ms/step - loss: 0.5639 - accuracy: 0.7060 - val_loss: 0.5597 - val_accuracy: 0.7025
Epoch 5/10
225/225 [==============================] - 18s 79ms/step - loss: 0.5272 - accuracy: 0.7296 - val_loss: 0.5407 - val_accuracy: 0.7063
Epoch 6/10
225/225 [==============================] - 17s 75ms/step - loss: 0.5044 - accuracy: 0.7504 - val_loss: 0.5334 - val_accuracy: 0.7312
Epoch 7/10
225/225 [==============================] - 17s 77ms/step - loss: 0.4703 - accuracy: 0.7713 - val_loss: 0.5226 - val_accuracy:

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (32, 48, 48, 64)          1792      
                                                                 
 max_pooling2d (MaxPooling2D  (32, 24, 24, 64)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 22, 22, 64)          36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 11, 11, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 9, 9, 64)            36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (32, 4, 4, 64)           0

In [12]:
test_dir = 'test_data'
target_df = pd.read_csv('test_labels.csv')


In [45]:
def preprocess_testing_data():
    testing_data = []
    for img_path in os.listdir(test_dir):
        label = label_test_image(img_path.split('.')[0])
        image = cv2.resize(cv2.imread(os.path.join(test_dir,img_path),cv2.IMREAD_UNCHANGED),(IMG_SIZE,IMG_SIZE))
        testing_data.append([image,label])
    shuffle(testing_data)
    return testing_data

In [43]:
def label_test_image(img):
    result = target_df.loc[target_df['id']==int(img)]['label']
    if result.tolist()[0] == 'cat':
        return 0
    else:
        return 1

In [54]:
testing_data = preprocess_testing_data()
test_imgs = []
test_labels = []
for features,label in testing_data:
    test_imgs.append(features)
    test_labels.append(label)

test_imgs = np.array(test_imgs)/255
test_labels = np.array(test_labels)

In [62]:
predictions = np.argmax(model.predict(test_imgs), axis=1)

In [64]:
test_labels

array([0, 0, 1, ..., 0, 0, 0])

In [61]:
predictions

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [65]:
model.evaluate(test_imgs,test_labels)

63/63 [==============================] - 2s 22ms/step - loss: 0.5703 - accuracy: 0.7242


[0.570263147354126, 0.7242063283920288]

In [77]:
count = 0
for i in range(2000):
    if predictions[i] == test_labels[i]:
        count += 1
print(count*100/2000,"%")

73.0 %
